In [1965]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
sns.set(rc={'figure.figsize':(11, 4)})

In [2104]:
'''MAIN DATA'''
data=pd.read_csv('train.csv')
data_1=data.drop(['Secondary Sales Quantity Returned','Secondary Sales Amount Invoiced','Secondary Sales Amount STC','Secondary Sales Amount Returned'],axis=1)
data_1.sort_values(by=['DATE'],inplace=True)
data_2=data_1
data_1.head()

# print(data_1.shape)


,SKU Code,CFA Code,Channel,Quantity,DATE,Secondary Sales Quantity Invoiced,SKU Combination
7250,16,11,2,6,2018-01-01,1962.0,424
6213,15,14,1,6,2018-01-01,399.0,375
21513,31,11,1,0,2018-01-01,1086.0,1179
6189,15,13,2,6,2018-01-01,6956.0,374
21536,31,11,2,0,2018-01-01,26385.0,1180


In [2105]:
'''PROMOTIONAL DATA'''
p_data=pd.read_csv('Promotional_Data.csv')
p_data.sort_values(by=['DATE'],inplace=True)
p_data2=p_data
p_data3=p_data
p_data2.head()
# promo_label={}

,DB Channel,SKU Code,DATE,Promo
0,All,31,2018-01-01,14
21,All,16,2018-01-01,15
22,All,17,2018-01-01,15
23,All,33,2018-01-01,15
24,All,42,2018-01-01,14


In [2106]:
'''TEST DATA'''
test=pd.read_csv('test.csv')
print(test.shape)
test.drop_duplicates(subset=['DATE','CFA Code','Channel','SKU Code'],inplace=True)
test_output=test
test.head()

(2609, 6)


,SKU Code,CFA Code,Channel,Quantity,DATE,SKU Combination
0,8,0,1,2,2020-01-01,0
1,8,0,1,2,2020-02-01,0
2,8,0,2,2,2020-01-01,1
3,8,0,2,2,2020-02-01,1
4,8,1,2,2,2020-01-01,3


In [2107]:
'''CHECKING DATES AND UNIQUE VALUES'''
dates_train=data['DATE']
dates_test=test['DATE']
dates_pro=p_data['DATE']
dates_train.unique()
data['SKU Code'].unique()
data['CFA Code'].unique()
data.Channel.unique()
p_data['DB Channel'].unique()
dates_pro.unique()
dates_test.unique()

array(['2020-01-01', '2020-02-01'], dtype=object)

In [2108]:
'''FOR MERGING PRO AND DATA [Expandin promotional dataset]'''
a0=p_data2.loc[p_data2['DB Channel'] == 'All']
a1=p_data2.loc[p_data2['DB Channel'] == 'All']
a2=p_data2.loc[p_data2['DB Channel'] == 'All']
a3=p_data2.loc[p_data2['DB Channel'] == '1&0 only']
a4=p_data2.loc[p_data2['DB Channel'] == '1&0 only']
a5=p_data2.loc[p_data2['DB Channel'] == '2 only']
a0['DB Channel'].replace({'All': '0'},inplace=True)
a1['DB Channel'].replace({'All': '1'},inplace=True)
a2['DB Channel'].replace({'All': '2'},inplace=True)
a3['DB Channel'].replace({'1&0 only': '0'},inplace=True)
a4['DB Channel'].replace({'1&0 only': '1'},inplace=True)
a5['DB Channel'].replace({'2 only': '2'},inplace=True)
frame0=[a0,a1,a2,a3,a4,a5]

C:\Users\Apurv_SJ\anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [2109]:
a6=pd.concat(frame0)
a6.sort_values(by='DATE',inplace=True)
a6.rename(columns = {'DB Channel':'Channel'}, inplace = True)
a7=a6[a6.DATE !='2020-01-01']
a7=a6[a6.DATE !='2020-02-01']
print(a7.dtypes)
a7['Channel']=a7['Channel'].astype(int)
a7.shape

Channel     object
SKU Code     int64
DATE        object
Promo        int64
dtype: object


C:\Users\Apurv_SJ\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


(2650, 4)

In [2110]:
a7.dtypes

Channel      int32
SKU Code     int64
DATE        object
Promo        int64
dtype: object

In [2111]:
data_2.drop_duplicates(subset=['DATE','CFA Code','Channel','SKU Code'],inplace=True)
data_2.shape

(31275, 7)

In [2112]:
d_p_m=pd.merge(data_2,a7,how='left',on=['SKU Code','Channel','DATE'])
d_p_m.drop_duplicates(subset=['DATE','CFA Code','Channel','SKU Code'],inplace=True)
d_p_m['Promo']=d_p_m['Promo'].fillna(0)
d_p_m.shape
d_p_m.isnull().any()

SKU Code                             False
CFA Code                             False
Channel                              False
Quantity                             False
DATE                                 False
Secondary Sales Quantity Invoiced    False
SKU Combination                      False
Promo                                False
dtype: bool

In [2113]:
scaled_comb=MinMaxScaler()
sc_comb=np.asarray(d_p_m['SKU Combination']).reshape(-1,1)
sc_comb=scaled_comb.fit_transform(sc_comb)
sc_comb[0]
d_p_m['SKU Combination']=sc_comb
d_p_m.head()

# scaled_tar=MinMaxScaler()
# sc_quant=np.asarray(d_p_m['Secondary Sales Quantity Invoiced']).reshape(-1,1)
# sc_quant=scaled_tar.fit_transform(sc_quant)
# sc_quant[0]
# d_p_m['Secondary Sales Quantity Invoiced']=sc_quant
# d_p_m.head()

,SKU Code,CFA Code,Channel,Quantity,DATE,Secondary Sales Quantity Invoiced,SKU Combination,Promo
0,16,11,2,6,2018-01-01,1962.0,0.217547,15.0
1,15,14,1,6,2018-01-01,399.0,0.192406,15.0
2,31,11,1,0,2018-01-01,1086.0,0.604926,14.0
3,15,13,2,6,2018-01-01,6956.0,0.191893,15.0
4,31,11,2,0,2018-01-01,26385.0,0.605439,14.0


In [2114]:
y=d_p_m['Secondary Sales Quantity Invoiced']
x=d_p_m.drop(['Secondary Sales Quantity Invoiced'],axis=1)
# x['SKU Combination']=0.1*x['SKU Combination']

In [2115]:
print(y.shape)
x.set_index('DATE',inplace=True)

(31275,)


In [2116]:
# x.drop(['SKU Combination'],axis=1,inplace=True)
# x['Prmomo']=[1 if i>0 else 0 for i in x['Promo']]
x.head()

,SKU Code,CFA Code,Channel,Quantity,SKU Combination,Promo
DATE,,,,,,
2018-01-01,16,11,2,6,0.217547,15.0
2018-01-01,15,14,1,6,0.192406,15.0
2018-01-01,31,11,1,0,0.604926,14.0
2018-01-01,15,13,2,6,0.191893,15.0
2018-01-01,31,11,2,0,0.605439,14.0


In [2117]:
y.head()
x['SKU Code']=x['SKU Code']+x['Promo']
# x['Channel']=x['Channel']+x['Promo']
# x['CFA Code']=x['CFA Code']+x['Quantity']
x.drop(['Promo'],axis=1,inplace=True)


In [2118]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,y)

In [2119]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest
import xgboost as xgb

In [2120]:
para_grids = {
            "n_estimators" : [100],
            "max_features" : ["auto"],
            "bootstrap"    : [True, False]
        }

In [2094]:
# lr=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
#            max_features='sqrt', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=2, min_samples_split=3,
#            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=7,
#            oob_score=False, random_state=None, verbose=0, warm_start=False)
lr=GradientBoostingRegressor(learning_rate=0.25,n_estimators=3500,
                             min_samples_split =25,min_samples_leaf = 26,max_depth =10,
                            subsample = 0.8,loss='ls',criterion='mse')
# lr=xgb.XGBRegressor(colsample_bytree=0.1,
#                  gamma=0,                 
#                  learning_rate=0.25,
#                  max_depth=10,
#                  min_child_weight=7.5,
#                  n_estimators=3500,                                                                    
#                  reg_alpha=0.75,
#                  reg_lambda=0.45,
#                  subsample=0.8,
#                  seed=42)
# lr=IsolationForest( n_estimators=450,
#     max_samples=12,
#     contamination='auto',
#     max_features=1.0,
#     bootstrap=True)
# grid = GridSearchCV(estimator, para_grids, scoring = 'mean_squared_error')
# grid.fit(Inputs_Treino, Outputs_Treino)
lr.fit(train_x,train_y)
lr_pred=lr.predict(test_x)
mean_squared_error(test_y,lr_pred)


17272648.68832953

In [2095]:
15157758.127391502



15157758.127391502

In [2121]:
'''FOR TEST(SUBMISSION) DATA'''
a8=a6[a6.DATE == '2020-01-01']
a9=a6[a6.DATE == '2020-02-01']
frame2=[a8,a9]
a10=pd.concat(frame2)
a10['Channel']=a10['Channel'].astype(int)
a10.head()
# a10.DATE.unique()


,Channel,SKU Code,DATE,Promo
904,0,12,2020-01-01,15
896,0,24,2020-01-01,12
918,1,9,2020-01-01,19
920,1,46,2020-01-01,19
921,1,47,2020-01-01,19


In [2122]:
dates_test.unique()

array(['2020-01-01', '2020-02-01'], dtype=object)

In [2123]:
test.head()

,SKU Code,CFA Code,Channel,Quantity,DATE,SKU Combination
0,8,0,1,2,2020-01-01,0
1,8,0,1,2,2020-02-01,0
2,8,0,2,2,2020-01-01,1
3,8,0,2,2,2020-02-01,1
4,8,1,2,2,2020-01-01,3


In [2124]:
test.shape

(2609, 6)

In [2125]:
test_merge=pd.merge(test,a10,how='left',on=['DATE','SKU Code','Channel'])
test_merge.drop_duplicates(subset=['DATE','CFA Code','Channel','SKU Code'],inplace=True)
test_merge['Promo']=test_merge['Promo'].fillna(0)
test_merge.set_index('DATE',inplace=True)
test_merge.shape
test_merge.head()


,SKU Code,CFA Code,Channel,Quantity,SKU Combination,Promo
DATE,,,,,,
2020-01-01,8,0,1,2,0,17.0
2020-02-01,8,0,1,2,0,17.0
2020-01-01,8,0,2,2,1,17.0
2020-02-01,8,0,2,2,1,17.0
2020-01-01,8,1,2,2,3,17.0


In [2126]:
test_merge.isnull().any()

sc_test=np.asarray(test_merge['SKU Combination']).reshape(-1,1)
sc_test=scaled_comb.fit_transform(sc_test)
test_merge['SKU Combination']=sc_test
test_merge['SKU Code']=test_merge['SKU Code']+test_merge['Promo']
# test_merge['Channel']=test_merge['Channel']*test_merge['Promo']
# test_merge['SKU Code']=test_merge['SKU Code']*test_merge['CFA Code']                                                         
test_merge.drop(['Promo'],inplace=True,axis=1)

print(test_merge.head())

            SKU Code  CFA Code  Channel  Quantity  SKU Combination
DATE                                                              
2020-01-01      25.0         0        1         2         0.000000
2020-02-01      25.0         0        1         2         0.000000
2020-01-01      25.0         0        2         2         0.000513
2020-02-01      25.0         0        2         2         0.000513
2020-01-01      25.0         1        2         2         0.001539


In [2127]:
rf_sub_pred=lr.predict(test_merge)

In [2128]:
# rf_sub_pred=np.asarray(rf_sub_pred).reshape(-1,1)
# rf_sub_pred=scaled_tar.inverse_transform(rf_sub_pred)

In [2129]:
test_output['Secondary Sales Quantity Invoiced']=rf_sub_pred
test_output.set_index('DATE',inplace=True)

In [2130]:
output=test_output.drop(['CFA Code','SKU Code','Channel','Quantity'],axis=1)
output.head()

,SKU Combination,Secondary Sales Quantity Invoiced
DATE,,
2020-01-01,0,438.293846
2020-02-01,0,438.293846
2020-01-01,1,24639.715297
2020-02-01,1,24639.715297
2020-01-01,3,8708.150901


In [2131]:
output.to_csv('output.csv')

In [2090]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor

seed = 7
np.random.seed(seed)

# Model
model = Sequential()
model.add(Dense(200, input_dim=36, kernel_initializer='normal', activation='relu'))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(25, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
# Compile model
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adadelta())

feature_cols = training_set[FEATURES]
labels = training_set[LABEL].values

model.fit(np.array(feature_cols), np.array(labels), epochs=100, batch_size=10)

NameError: name 'keras' is not defined